# Horse Racing Prediction by Deep Learning

This AI Horse racing prediction is using a 3-layer Neural Network to predict the finishing position of each horse in a race. The data used to train this model is obtained from the Hong Kong Jockey Club (HKJC). The model is built and trained by Tensorflow. Supervised learning is used in this project to classify the expected finishing position of the horses.

<H2> Part 1: Data Input and Preprocessing </H2>

In this part of the program, we will import the data obtained from the HKJC. First of all, the following features were selected based on my past horse picking experience, namely:

- position: The starting position of the horse. If the position is "1", it indicates the closest position to the hurdle and should be benficial in non-straight race courses.

- load: This is the loading of the horse in pounds. Maximum is 133.

- ON odds: This is the overnight odds of the horse provided by the HKJC.

- odds: This is the odds of the horse 15 min before the race.

- class: This is the class of the case. It is common to all horses in a race except special races.

- num horses: This is the number of horses participated in the race.


In [1]:
#Loading the data and preprocessing it.

import pandas as pd
import tensorflow as tf
tf.__version__

PATH_TRAINING_DATA = 'training_data/horse_data_train_test.csv'

dataset = pd.read_csv(PATH_TRAINING_DATA)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle=False)


In [2]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) #use "fit_transform" for training data
X_test = sc.transform(X_test)       #use "transform" for testing data

y_train = tf.keras.utils.to_categorical((y_train-1), 14)
y_test = tf.keras.utils.to_categorical((y_test-1), 14)

print(X_train)

[[-0.11982255  0.8075992  -0.42994503 -0.65187705 -0.92180607 -1.19613971]
 [-0.61764202 -0.6554524  -0.75281787 -0.86161937 -0.92180607 -1.19613971]
 [ 0.62690665 -1.05446647 -0.55412689 -0.67984269 -0.92180607 -1.19613971]
 ...
 [-0.86655175 -0.78845709 -0.80249061 -0.88958501  0.52931833 -0.30839698]
 [ 1.12472611 -0.92146178  0.03573695 -0.20442677  0.52931833 -0.30839698]
 [-1.61328094 -0.92146178 -0.58517236 -0.67984269  0.52931833 -0.30839698]]


<H2> Part 2: Building the Model </H2>

We will construct a basic 3-layer model: one input layer, one output layer and one hidden layer.
I have tried relu and sigmoid as the activation, and found that sigmoid seems to be a better choice for this project.


In [3]:
# Initializing the ANN
model = tf.keras.models.Sequential()

# Adding the input layer and the first layer
# We have six features now, and one bias term, so the input layer has a size of 6+1
model.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))

# Adding the input layer and the hidden layer
# We set the size of the hidden layer to be the mean of input and output later, i.e. 10
#model.add(tf.keras.layers.Dense(units=6, activation='sigmoid')) 
model.add(tf.keras.layers.Dense(units=10, activation='sigmoid')) 

# Adding a drop out layer
model.add(tf.keras.layers.Dropout(0.2))

#model.add(tf.keras.layers.Dense(units=6, activation='sigmoid')) 
#model.add(tf.keras.layers.Dropout(0.2))

# Adding the output layer
# We have 14 outputs, so the output later has a size of 14
model.add(tf.keras.layers.Dense(units=14, activation='softmax'))

# Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

<H2> Part 3a: [Optional] Training and saving a new Model </H2>

In this section, we would either traing the model from scratch or just load a pre-trained model that is shipped with this Jupyter notebook.

- If the training data has been changed or the model has been renewed, go to train a new model in code cell 3a)
- If you want to save some time, just skip cell 3a) and load a pre-trained model in code cell 3b)


In [ ]:
#This is code cell 3a) that trains a model from scratch and then saves it. Please note that it takes hours to train!

import tensorflow as tf
import datetime, os

%load_ext tensorboard

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

print("rows in X_train = " + str(X_train.shape[0]) )
print("rows in y_train = " + str(y_train.shape[0]) )

print("Training model...")
model.fit(x=X_train, 
          y=y_train, 
          batch_size = 14, 
          epochs = 10000, #30000 #50000 #20000
          validation_data=(X_test, y_test), 
          callbacks=[tensorboard_callback])

# save the model for later use
print("Saving model...")
model.save('saved_model/my_model')

print("Model saved!")

rows in X_train = 403
rows in y_train = 403
Training model...
Epoch 1/10000
29/29 [==============================] - 2s 29ms/step - loss: 2.7463 - accuracy: 0.0720 - val_loss: 2.7153 - val_accuracy: 0.0693
Epoch 2/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.7280 - accuracy: 0.0695 - val_loss: 2.6934 - val_accuracy: 0.0693
Epoch 3/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.6978 - accuracy: 0.0769 - val_loss: 2.6788 - val_accuracy: 0.0693
Epoch 4/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.6960 - accuracy: 0.0769 - val_loss: 2.6686 - val_accuracy: 0.0693
Epoch 5/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.6736 - accuracy: 0.0819 - val_loss: 2.6608 - val_accuracy: 0.0693
Epoch 6/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.6817 - accuracy: 0.0496 - val_loss: 2.6553 - val_accuracy: 0.0693
Epoch 7/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.65

29/29 [==============================] - 0s 4ms/step - loss: 2.4901 - accuracy: 0.1563 - val_loss: 2.4780 - val_accuracy: 0.1287
Epoch 58/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.4928 - accuracy: 0.1340 - val_loss: 2.4734 - val_accuracy: 0.1287
Epoch 59/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.4959 - accuracy: 0.1290 - val_loss: 2.4702 - val_accuracy: 0.1386
Epoch 60/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.4637 - accuracy: 0.1588 - val_loss: 2.4657 - val_accuracy: 0.1386
Epoch 61/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.4872 - accuracy: 0.1290 - val_loss: 2.4609 - val_accuracy: 0.1386
Epoch 62/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.4711 - accuracy: 0.1166 - val_loss: 2.4573 - val_accuracy: 0.1287
Epoch 63/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.4783 - accuracy: 0.1191 - val_loss: 2.4531 - val_accuracy: 0.1287
Epoch 6

Epoch 114/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.3779 - accuracy: 0.1836 - val_loss: 2.3769 - val_accuracy: 0.0990
Epoch 115/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.3633 - accuracy: 0.1538 - val_loss: 2.3764 - val_accuracy: 0.0990
Epoch 116/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.3775 - accuracy: 0.1588 - val_loss: 2.3749 - val_accuracy: 0.1089
Epoch 117/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.3743 - accuracy: 0.1489 - val_loss: 2.3750 - val_accuracy: 0.0990
Epoch 118/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.3635 - accuracy: 0.1514 - val_loss: 2.3739 - val_accuracy: 0.0990
Epoch 119/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.3682 - accuracy: 0.1588 - val_loss: 2.3732 - val_accuracy: 0.1089
Epoch 120/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.3635 - accuracy: 0.1563 - val_loss: 2.3724 - val_ac

29/29 [==============================] - 0s 3ms/step - loss: 2.3310 - accuracy: 0.1489 - val_loss: 2.3531 - val_accuracy: 0.1485
Epoch 171/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.3233 - accuracy: 0.1687 - val_loss: 2.3532 - val_accuracy: 0.1485
Epoch 172/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.3222 - accuracy: 0.1687 - val_loss: 2.3530 - val_accuracy: 0.1485
Epoch 173/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.3067 - accuracy: 0.1514 - val_loss: 2.3532 - val_accuracy: 0.1485
Epoch 174/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.3291 - accuracy: 0.1514 - val_loss: 2.3531 - val_accuracy: 0.1485
Epoch 175/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.3262 - accuracy: 0.1365 - val_loss: 2.3518 - val_accuracy: 0.1485
Epoch 176/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.3161 - accuracy: 0.1538 - val_loss: 2.3516 - val_accuracy: 0.1485
E

29/29 [==============================] - 0s 5ms/step - loss: 2.3112 - accuracy: 0.1935 - val_loss: 2.3548 - val_accuracy: 0.1485
Epoch 227/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.2910 - accuracy: 0.1960 - val_loss: 2.3557 - val_accuracy: 0.1485
Epoch 228/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.2835 - accuracy: 0.1911 - val_loss: 2.3563 - val_accuracy: 0.1386
Epoch 229/10000
29/29 [==============================] - 0s 5ms/step - loss: 2.2883 - accuracy: 0.1886 - val_loss: 2.3564 - val_accuracy: 0.1485
Epoch 230/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.2874 - accuracy: 0.1762 - val_loss: 2.3581 - val_accuracy: 0.1485
Epoch 231/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.2837 - accuracy: 0.1712 - val_loss: 2.3571 - val_accuracy: 0.1386
Epoch 232/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.2705 - accuracy: 0.1762 - val_loss: 2.3566 - val_accuracy: 0.1485
E

...

29/29 [==============================] - 0s 4ms/step - loss: 2.0285 - accuracy: 0.2928 - val_loss: 3.1096 - val_accuracy: 0.1188
Epoch 2915/10000
29/29 [==============================] - 0s 3ms/step - loss: 1.9901 - accuracy: 0.2878 - val_loss: 3.1118 - val_accuracy: 0.1089
Epoch 2916/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.0257 - accuracy: 0.2804 - val_loss: 3.1117 - val_accuracy: 0.1188
Epoch 2917/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.0170 - accuracy: 0.2705 - val_loss: 3.1119 - val_accuracy: 0.1188
Epoch 2918/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.0027 - accuracy: 0.2854 - val_loss: 3.1129 - val_accuracy: 0.1188
Epoch 2919/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.0258 - accuracy: 0.2928 - val_loss: 3.1105 - val_accuracy: 0.1089
Epoch 2920/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.0078 - accuracy: 0.2680 - val_loss: 3.1084 - val_accuracy: 0.

Epoch 2970/10000
29/29 [==============================] - 0s 4ms/step - loss: 1.9967 - accuracy: 0.3052 - val_loss: 3.1356 - val_accuracy: 0.1188
Epoch 2971/10000
29/29 [==============================] - 0s 4ms/step - loss: 1.9819 - accuracy: 0.2953 - val_loss: 3.1318 - val_accuracy: 0.1287
Epoch 2972/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.0022 - accuracy: 0.2953 - val_loss: 3.1323 - val_accuracy: 0.1386
Epoch 2973/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.0177 - accuracy: 0.2878 - val_loss: 3.1372 - val_accuracy: 0.1287
Epoch 2974/10000
29/29 [==============================] - 0s 3ms/step - loss: 1.9787 - accuracy: 0.2854 - val_loss: 3.1377 - val_accuracy: 0.1287
Epoch 2975/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.0000 - accuracy: 0.3002 - val_loss: 3.1373 - val_accuracy: 0.1287
Epoch 2976/10000
29/29 [==============================] - 0s 4ms/step - loss: 1.9953 - accuracy: 0.2754 - val_loss: 3.1402 -

29/29 [==============================] - 0s 3ms/step - loss: 2.0115 - accuracy: 0.2854 - val_loss: 3.1588 - val_accuracy: 0.1188
Epoch 3027/10000
29/29 [==============================] - 0s 4ms/step - loss: 1.9979 - accuracy: 0.2779 - val_loss: 3.1596 - val_accuracy: 0.1287
Epoch 3028/10000
29/29 [==============================] - 0s 4ms/step - loss: 1.9810 - accuracy: 0.3002 - val_loss: 3.1590 - val_accuracy: 0.1386
Epoch 3029/10000
29/29 [==============================] - 0s 4ms/step - loss: 1.9890 - accuracy: 0.2978 - val_loss: 3.1628 - val_accuracy: 0.1386
Epoch 3030/10000
29/29 [==============================] - 0s 3ms/step - loss: 1.9919 - accuracy: 0.2903 - val_loss: 3.1630 - val_accuracy: 0.1188
Epoch 3031/10000
29/29 [==============================] - 0s 3ms/step - loss: 1.9903 - accuracy: 0.2978 - val_loss: 3.1667 - val_accuracy: 0.1287
Epoch 3032/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.0078 - accuracy: 0.2680 - val_loss: 3.1643 - val_accuracy: 0.

29/29 [==============================] - 0s 3ms/step - loss: 1.9721 - accuracy: 0.2978 - val_loss: 3.1838 - val_accuracy: 0.1188
Epoch 3083/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.0253 - accuracy: 0.2481 - val_loss: 3.1825 - val_accuracy: 0.1089
Epoch 3084/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.0103 - accuracy: 0.2730 - val_loss: 3.1852 - val_accuracy: 0.1287
Epoch 3085/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.0233 - accuracy: 0.2680 - val_loss: 3.1821 - val_accuracy: 0.1287
Epoch 3086/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.0140 - accuracy: 0.2878 - val_loss: 3.1839 - val_accuracy: 0.1287
Epoch 3087/10000
29/29 [==============================] - 0s 3ms/step - loss: 1.9964 - accuracy: 0.2903 - val_loss: 3.1854 - val_accuracy: 0.1287
Epoch 3088/10000
29/29 [==============================] - 0s 3ms/step - loss: 1.9774 - accuracy: 0.3102 - val_loss: 3.1895 - val_accuracy: 0.

29/29 [==============================] - 0s 4ms/step - loss: 2.0068 - accuracy: 0.2903 - val_loss: 3.2063 - val_accuracy: 0.1287
Epoch 3139/10000
29/29 [==============================] - 0s 4ms/step - loss: 1.9960 - accuracy: 0.2953 - val_loss: 3.2056 - val_accuracy: 0.1188
Epoch 3140/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.0254 - accuracy: 0.2705 - val_loss: 3.2123 - val_accuracy: 0.1188
Epoch 3141/10000
29/29 [==============================] - 0s 4ms/step - loss: 2.0193 - accuracy: 0.2978 - val_loss: 3.2091 - val_accuracy: 0.1188
Epoch 3142/10000
29/29 [==============================] - 0s 3ms/step - loss: 2.0079 - accuracy: 0.2779 - val_loss: 3.2083 - val_accuracy: 0.1188
Epoch 3143/10000
26/29 [=========================>....] - ETA: 0s - loss: 1.9830 - accuracy: 0.3022

<H2> Part 3b: Loading a pre-trained Model </H2>

If you have ever run cell 3a), you will load the model that you have trained. Otherwise, you will load a pre-trained model that is shipped with this Jupyter notebook.

In [ ]:
#This is code cell 3b) that loads a pre-trained model. 

model = tf.keras.models.load_model('saved_model/my_model')

print("model is loaded")

<H2> Part 4: Evaluation </H2>

This part calculates the confusion matrix and accuracy_score of the model.

In [ ]:
# Restore the weights
import numpy as np
import pandas as pd

dataset = pd.read_csv('training_data/horse_data_train_test.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


import tensorflow as tf
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Predicting the Test set results
y_pred = model.predict( X_test )  #y_pred = new_model.predict( sc.fit_transform(X_test) ) 
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, (y_pred+1))
print("cm=", cm)
print("accuracy_score=", accuracy_score(y_test, (y_pred+1)))

<H2> Part5: Visualization of the model by Tensorboard </H2>

By making use of the powerful visual tools given by Tensorboard, we can tune the hyper parameters of the model and visualize the results easily. If the epoch_accuracy is increasing steadily with epoch for the validation set, we could be confident that the model is on track. 

If we are not satisfy with the performaces shown in the Tensorboard, we can go back to refine the data processing, model building and hyper parameters and check on Tensorboard again iteratively.

In [ ]:
print("Calling tensorboard...")

%tensorboard --logdir logs


<H2> Part6: Prediction </H2>

We need to create a new csv file that contains the data of a new race in the folder new_data. Then we run the following two cells to get the results.

In [ ]:
#This is the csv containing the data of a new race:

#CURRENT_RACE_DATA = 'new_data/horse_data_20210919_race7.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20210922_race2.csv'
CURRENT_RACE_DATA = 'new_data/horse_data_20210922_race3.csv'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

PATH_TRAINING_DATA = 'training_data/horse_data_train_test.csv'

#training data
dataset = pd.read_csv(PATH_TRAINING_DATA)
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0, shuffle=False)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)

#live data
dataset = pd.read_csv(CURRENT_RACE_DATA)
X_live = dataset.iloc[:, :-1].values

#use "transform" for live data
X_live = sc.transform(X_live)

#loading pretrained model
model = tf.keras.models.load_model('saved_model/my_model')

# Predicting the Test set results
y_pred = model.predict(X_live)  

# Predicting the finishing position
y_pred_finishing = np.argmax(y_pred, axis = 1) 
print("Expected finishing positions=", y_pred_finishing+1)

#print("The real finishing positions= [ 6  10 9  5  2  7  8  1  3  4] ")